In [1]:
import jsonlines
import re
from tqdm import tqdm
from database import config
import mysql.connector
import binascii
db = mysql.connector.connect(**config)
cursor = db.cursor()


In [11]:
with jsonlines.open("/home/ubuntu/projects/SHIP_search/corpus_late.jsonl", 'r') as reader:
    for line in tqdm(reader):
        print(line)
        break
    # {'93518974_0': {'createtime': '2019-01-01 22:51:00', 'url': 'https://www.donga.com/news/Politics/article/all/20190101/93518974/1', 'thumburl': 'https://dimg.donga.com/wps/NEWS/IMAGE/2019/01/01/93518865.2.jpg', 'source': '
    # createtime, url, thumburl, source, ctx

0it [00:00, ?it/s]

{'93518974_0': {'createtime': '2019-01-01 22:51:00', 'url': 'https://www.donga.com/news/Politics/article/all/20190101/93518974/1', 'thumburl': 'https://dimg.donga.com/wps/NEWS/IMAGE/2019/01/01/93518865.2.jpg', 'source': '뉴스1', 'ctx': '<제목> 이낙연 총리 "대기업 사업·대형 공공인프라 구축 의욕적으로" <작성일> 2019년 1월 1일 <본문> 이낙연 국무총리는 1일 집권 3년 차에 접어드는 2019년 경제 활성화를 위해 대기업과 민간기업의 대규모 투자 사업이 조기에 착공하도록 지원하고 광역권 교통·물류 등 대형 공공인프라 구축 등 투자를 의욕적으로 할 계획이라고 밝혔다.이 총리는 이날 기해년 첫 일정으로 KBS 뉴스9에 출연해 \'집권 3년 차 성과를 내기 위한 특단의 대책은 무엇인가\'라는 질문에 이같이 말했다.이 총리는 "경제 활성화를 위해서는 투자를 살려야 한다"며 "현재 제일 부진한 것이 투자와 고용"이라고 진단했다.이어 고용 부분에 대해 "일부 내용에 논란이 있긴 하지만 일자리 지표가 최근 2~3개월 동안 개선이 되고 있다"며 "새해에는 일하더라도 소득이 낮은 분들에게 드리는 근로장려세는 약 5조원을, 일자리 예산은 23조원을 편성했다. 이런 예산들을 효율적으로 집행하면 고용 개선 효과가 나타날 것"이라고 내다봤다.이 총리는 집권 3년 차에도 경제정책 기조를 바꾸기보다 현장의 부작용이 없는지 살피고 수용성을 높이기 위해 유연하게 할 필요가 있는지 살피겠다고 말했다.이어 경제 정책에 대한 부정적 평가의 이유를 부작용 문제로 보냐는 질문에는 "현장의 수용성이나 부작용은 분명히 있다. 그런 것들에 대해서 일부러 눈을 감거나 해서는 안 된다"며 "있는 그대로를 직시하고 현장에서 많이 수용되도록, 부작용을 최소화하도록 정책 이행 과정에서 국민의 고통을 최소화하는 노력을 배가해야

In [3]:

#### 오리지날.  
with jsonlines.open("/home/ubuntu/projects/SHIP_search/corpus_late.jsonl", 'r') as reader:
    for line in tqdm(reader):
        gid, item = list(line.items())[0]
        ctx=item['ctx'] 
        title = re.search(r'(?<=\<제목\>).+(?=\<작성일\>)',ctx)[0].strip()
        cursor.execute(f"""insert ignore into news_gpt.news_recent values(
        "{gid}",
        "{item['createtime']}",
        %s,
        %s,
        %s,
        "{item['url']}",
        "{item['thumburl']}",
        "{item['source']}",
        NULL,
        NULL
        )""", (title.replace(' ',''), title, ctx.encode('utf-8')))
db.commit()

0it [00:00, ?it/s]

545092it [02:37, 3453.64it/s]


In [2]:
# #### 전부 +  종합   # 이렇게 굳이 안하기로 ********
# import pickle
# import json
# with jsonlines.open("/home/ubuntu/projects/SHIP_search/corpus_late.jsonl", 'r') as reader:
#     for line in tqdm(reader):
#         gid, item = list(line.items())[0]
#         ctx=item['ctx'] 
#         title = re.search(r'(?<=\<제목\>).+(?=\<작성일\>)',ctx)[0].strip()
#         cursor.execute(f"""insert ignore into news_gpt.news_recent values(
#         "{gid}",
#         "{item['createtime']}",
#         %s,
#         %s,
#         %s,
#         "{item['url']}",
#         "{item['thumburl']}",
#         "{item['source']}",
#         NULL,
#         NULL,
#         %s
#         )""", (title.replace(' ',''), title, ctx.encode('utf-8'), pickle.dumps(item)) )
# db.commit()

545092it [03:52, 2339.77it/s]


In [4]:
import pickle

with open('shard_0', 'rb') as f:
    shard0 = dict(pickle.load(f))

cursor.execute("""
select gid from news_gpt.news_recent;
""")
gids = cursor.fetchall()    

In [6]:
import numpy as np
from tqdm import tqdm

for gid in tqdm(gids):
    gid = gid[0]
    vec = shard0[f'gid:{gid}']
    
    cursor.execute(f"""
    update news_gpt.news_recent set vec =%s where gid="{gid}"
    """,(vec.tobytes(),))
db.commit()
# shard0['gid:93518974_0']    

100%|██████████| 545092/545092 [05:22<00:00, 1690.05it/s]


: 

In [ ]:
cursor.execute("""
select gid, vec from news_gpt.news_recent;
""")
temp = cursor.fetchall()    
# gid 다 가져오는데 1분 내외   / 500k 에 1G
# memory에 올라와있으면 1초

In [ ]:
import numpy as np
cursor.execute("""
select gid, vec from news_gpt.news_recent;
""")
dics = {}
for gid, vec in  cursor.fetchall():
    dics[gid] = np.frombuffer(vec, dtype='float32')

In [ ]:
dics['100000066_0'].shape

(768,)

In [ ]:
###################################################################################
# corpus_old 올리기

In [3]:
with jsonlines.open("/home/ubuntu/projects/SHIP_search/corpus_old.jsonl", 'r') as reader:
    for line in tqdm(reader):
        print(line)
        break

0it [00:00, ?it/s]

{'7897497_0': {'createtime': '2003-01-01 01:41:00', 'title': "'대통령 사면권 남용' 현직부장판사 공개비판", 'url': 'https://www.donga.com/news/Society/article/all/20030101/7897497/1', 'thumburl': '', 'source': '동아일보', 'cate_code': '03', 'ctx': '<제목> \'대통령 사면권 남용\' 현직부장판사 공개비판 <작성일> 2002년 12월 31일 <본문> 현직 부장판사가 지난해 12월30일 정부가 발표한 사면 조치를 비롯해 역대 대통령들의 사면권 남용을 비판하고 나섰다. 수원지법 성남지원 이충상(이충상) 부장판사는 지난해 12월31일 법원 내부 통신망에 게재한 논문 \'사면법 개정안-사면권의 제한\'에서 이번 사면과 관련, "판결이 확정된 지 10일과 3개월 밖에 안된 김영재(김영재) 전 금융감독원 부원장보와 최일홍(최일홍) 전 체육진흥공단 이사장을 사면한 것은 너무 이르다는 생각이 든다"고 말했다. 이 부장판사는 "대통령의 사면권은 무제한이고, 역대 대통령들이 이를 남용했다는 공감대가 형성돼 있다"며 "판결문의 잉크가 채 마르기도 전에 사면을 하는 것을 막기 위해 원칙적으로 판결 확정일로부터 형기의 3분의 1을 경과하기 전에는 사면할 수 없도록 해야 한다"고 주장했다. 이 부장판사는 "대통령과 국회, 대법원장이 뽑은 이들로 구성된 사면심사회가 청구해야 특별사면이 가능하도록 하는 방안도 가능하다"며 "사면권이 제한되면 최소한 얼마간은 복역해야 하기 때문에 정치인 등에 대한 법원의 양형권도 회복될 수 있다"고 덧붙였다.'}}


In [2]:

with jsonlines.open("/home/ubuntu/projects/SHIP_search/corpus_old.jsonl", 'r') as reader:
    for line in tqdm(reader):
        gid, item = list(line.items())[0]
        ctx=item['ctx'] 
        cursor.execute(f"""insert ignore into news_gpt.news_old values(
        "{gid}",
        "{item['createtime']}",
        %s,
        %s,
        %s,
        "{item['url']}",
        "{item['thumburl']}",
        "{item['source']}",
        "{item['cate_code']}",
        NULL
        )""", (item['title'].replace(' ',''), item['title'], ctx.encode('utf-8')))
db.commit()

# corpus_old 는 vec 은 필요 없음

1097788it [05:47, 3159.60it/s]
